In [10]:
import pandas as pd
import numpy as np
import sklearn
from textgenrnn import textgenrnn
import emoji
from sklearn.linear_model import LogisticRegression
import re

In [61]:
def alias_translator(aliases):
	results = []
	BASE = ":"

	for alias in aliases:
		emo_boi = emoji.emojize(BASE + alias + BASE)
		results.append(emo_boi)
	return results

def emoji_counter(clean):
	ALCOHOL_ALIAS = ["tropical_drink", "wine_glass", "beer", "cocktail", "beers", "sake"]
	DRUG_ALIAS = ["smoking", "no_smoking", "pill", "dash", "leaves", "mushroom", "ear_of_rice"]
	alcohol_emojis = alias_translator(ALCOHOL_ALIAS)
	drug_emojis = alias_translator(DRUG_ALIAS)
	ILLICIT_EMOJIS = alcohol_emojis + drug_emojis
	count = 0
	for character in clean:
		if character in ILLICIT_EMOJIS:
			count += 1
	return count

def phrase_counter(clean):
	ALCOHOL_PHRASES = ["alc", "alcohol", "bubbly", "champagne", "drinks", "beer", "bud", "drank"]
	DRUG_PHRASES = ["weed", "pills", "ecstasy", "broccoli", "plug", "codeine", 
	"high", "buzzed", "stoned", "420", "smoke", "popper"]
	VAPE_PHRASES = ["pods", "pod", "juul", "suorin", "vape", "vaping", "vape"]
	BAD_BOIS = ALCOHOL_PHRASES + DRUG_PHRASES + VAPE_PHRASES
	count = 0 
	for word in clean:
		if word in BAD_BOIS:
			count += 1

	return count

In [51]:
df = pd.read_csv("train_test_data.csv")

In [52]:
df.tail()

payment_text  label
94                                      Mountain food      0
95  you are the bestest person ever and I apprecia...      0
96                        you’re doing great sweetie🥂      0
97                   I never paid you for gum yikenss      0
98                                             hi hey      0

In [71]:
def text_cleaner(dirty):
    clean = dirty.lower()
    return clean

# returns X_train, X_test, y_train, y_test
def pipeline(df):
    df["payment_text"] = df["payment_text"].apply(text_cleaner)
    df["emojis"] = df["payment_text"].apply(str.split).apply(emoji_counter)
    df["bad_phrases"] = df["payment_text"].apply(str.split).apply(phrase_counter)
    Y = df["label"]
    X = df[["emojis", "bad_phrases"]]
    return sklearn.model_selection.train_test_split(X, Y, test_size = 0.33)

In [5]:
def create_more(text_file):
    textgen = textgenrnn()
    textgen.train_from_file(text_file, num_epochs=10)
    return textgen.generate(5)

In [6]:
create_more('clean.txt')

48 texts collected.
Training on 998 character sequences.
Epoch 1/10
7/7 [==============================] - 3s 370ms/step - loss: 2.6060
####################
Temperature: 0.2
####################
I makiki bumbbie pro money person position but but bust intelding bust but housh bump you fool my bust but hey hell some pal bent bust the person pett

The budge fool mod

The budge but bust white

####################
Temperature: 0.5
####################
empty solving run of some Of y

Fook of my the plant

No food my bust my my buddie past

####################
Temperature: 1.0
####################
My googioo

d my bud iftiest on you white

Bbiti the tuthe the bumbbook

Epoch 2/10
7/7 [==============================] - 1s 206ms/step - loss: 1.6924
####################
Temperature: 0.2
####################
The brook

The sounddenoum of 1000

Appliviation but I’m deposit

####################
Temperature: 0.5
####################
The weekly grend heals and peace I’m so so so so so so so so so 

  0%|          | 0/5 [00:00<?, ?it/s]

Taken encer problem



 40%|████      | 2/5 [00:00<00:00,  3.55it/s]

Empt the bestest person 

The browz



...

of a bike



 80%|████████  | 4/5 [00:01<00:00,  3.27it/s]

out of a wein paid you for 1 month



100%|██████████| 5/5 [00:01<00:00,  3.39it/s]

Rental for 1.5 month



In [7]:
create_more('illicit.txt')

49 texts collected.
Training on 774 character sequences.
Epoch 1/10
6/6 [==============================] - 3s 485ms/step - loss: 2.7629
####################
Temperature: 0.2
####################
Brown drank

Berder for the gradk

Brown drank

####################
Temperature: 0.5
####################
Gribstend drank

rolling and drank

Brand those pills

####################
Temperature: 1.0
####################
reink

Bed for bedrock

i bostur_ weed in the dandy drank

Epoch 2/10
6/6 [==============================] - 2s 263ms/step - loss: 1.6603
####################
Temperature: 0.2
####################
Strippers

Banks and uber drank

Strippers

####################
Temperature: 0.5
####################
Herb

God per dog stuff

Drank to drank in the drank in the first drank

####################
Temperature: 1.0
####################
[f]

card daughters aboud bakog and me fougrabor user to weed is this eigners

the weed drank

Epoch 3/10
6/6 [==============================] - 2s 266m

  0%|          | 0/5 [00:00<?, ?it/s]

The Heroid drank

Drank



 20%|██        | 1/5 [00:00<00:02,  1.98it/s]

Pour up, weed anole and notess

drank



 60%|██████    | 3/5 [00:00<00:00,  2.56it/s]

Strippers



100%|██████████| 5/5 [00:01<00:00,  3.10it/s]

pods and alc & Strippers

Palm trees



In [16]:
def tokenize(sentences):
    words = []
    for sentence in sentences:
        w = word_extraction(sentence)
        words.extend(w)
        
    words = sorted(list(set(words)))
    return words

def word_extraction(sentence):
    ignore = ['a', "the", "is"]
    words = re.sub("[^\w]", " ",  sentence).split()
    cleaned_text = [w.lower() for w in words if w not in ignore]
    return cleaned_text    
    
def generate_bow(allsentences):    
    vocab = tokenize(allsentences)
    print("Word List for Document \n{0} \n".format(vocab));

    for sentence in allsentences:
        words = word_extraction(sentence)
        bag_vector = np.zeros(len(vocab))
        for w in words:
            for i,word in enumerate(vocab):
                if word == w: 
                    bag_vector[i] += 1